In [29]:
import os 
import pyttsx3
from ebooklib import epub
from bs4 import BeautifulSoup
import fitz
import tkinter as tk
from tkinter import filedialog, ttk


In [19]:
engine = pyttsx3.init()

In [20]:
# Set default properties
engine.setProperty('rate', 150)    # Speech rate
engine.setProperty('volume', 1.0)  # Volume (0.0 to 1.0)

In [21]:
voices = engine.getProperty('voices')

for index, voice in enumerate(voices):
    print(f"Voice {index}: {voice.name} - {voice.languages}")

Voice 0: Microsoft David Desktop - English (United States) - []
Voice 1: Microsoft Zira Desktop - English (United States) - []


In [22]:
## Function to read .txt files
def read_text_file(file_path):
    with open(file_path, 'r', encoding = 'utf-8') as file:
        return file.read()

In [23]:
## Function to read .epub files
def read_epub_file(file_path):
    book = epub.read_pub(file_path)
    text = "" 

    for doc in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
        content = doc.get_content().decode('utf-8')
        soup = BeautifulSoup(content, 'html.parser')
        text += soup.get_text()

    return text

In [24]:
# Function to read .pdf files
def read_pdf_file(file_path):
    pdf = fitz.open(file_path)
    text = ""

    for page in pdf:
        text += page.get_text()

    return text

In [25]:
# Playback Control Functions

def play_text(text):
    engine.say(text)
    engine.runAndWait()

def pause_reading():
    engine.pause()

def resume_reading():
    engine.resume()

def stop_reading():
    engine.stop()

In [26]:
# Speech Settings Functions 

def set_voice(voice_id):
    engine.setProperty('voice', voices[voice_id].id)

def set_rate(rate):
    engine.setProperty('rate', rate)

def set_volume(volume):
    engine.setProperty('volume', volume)

In [27]:
# Function to Save to Audiio File
def save_to_audio(text, filename):
    engine.save_to_file(text, filename)
    engine.runAndWait()

In [30]:
# Setup GUI
root = tk.Tk()
root.title("Audiobook Reader")

# Load Button
load_button = tk.Button(root, text="Load and Read Book", command=load_and_read)
load_button.pack(pady=10)

# Voice Selection
voice_label = tk.Label(root, text="Select Voice:")
voice_label.pack()
voice_combo = ttk.Combobox(root, values=[voice.name for voice in voices])
voice_combo.current(0)
voice_combo.bind("<<ComboboxSelected>>", change_voice)
voice_combo.pack(pady=5)

# Rate Slider
rate_label = tk.Label(root, text="Speech Rate:")
rate_label.pack()
rate_slider = tk.Scale(root, from_=100, to=200, orient=tk.HORIZONTAL, command=change_rate)
rate_slider.set(engine.getProperty('rate'))
rate_slider.pack(pady=5)

# Volume Slider
volume_label = tk.Label(root, text="Volume:")
volume_label.pack()
volume_slider = tk.Scale(root, from_=0, to=1, resolution=0.1, orient=tk.HORIZONTAL, command=change_volume)
volume_slider.set(engine.getProperty('volume'))
volume_slider.pack(pady=5)

root.mainloop()

NameError: name 'load_and_read' is not defined